# Booking WebScraper # 

This Notebooks contains the code to scrape the information off of some of Booking.com's Portugal accomodations.

This webscraper utilyzes the Selinum Framwork and the BS4 Library.<br>
We are using selenium beacuse booking.com's webpage is dynamicly loaded meaning we have to load the page to a brower to get the correct and complete html code. A simple http request using the request library is not enought to get the full html.<br>
We are using the BS4 to easiy scrape the data on the extracted html.



##  1. Importing all Necessary Libraries ##

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import glob
from bs4 import BeautifulSoup
from pandas import DataFrame, read_excel, concat

## 2. Setting up the options ##

In [2]:
# Webdriver laod options
options = Options()
options.headless = False # If False this will open the webdriver window. If True the scraper will operate in headless mode

## 3. Identifying the base URL to scrape the Top Destinations of Booking ##

### Important Note ###

Portugal has over 40K listings on Booking. There is no way to access to all listing since booking only loads up to 40 pages at a time (1000 listings.) <br>
Therefore, we decided to get only the data of Booking's Top Destinations' cities. If a city has more than 1000 listing we will only get the first 1000 that booking displays to us. <br>
Only 6 cities have more than 1000 accomodations listed on Booking: Lisbon, Porto, Albufeira, Portimão, Lagos and Funchal.

We identifyed the base URL that only changes based on the Top destiantions chosen. We will scrape the top destinations to then append to the base URL.

In [5]:
url = "https://www.booking.com/searchresults.en-us.html?ss=Portugal&ssne=Portugal&ssne_untouched=Portugal&label=gen173nr-1FCAEoggI46AdIM1gEaLsBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAK27smiBsACAdICJDAzNmNmODZkLTEzNWUtNGY4YS05NDRjLTA4NDNhOTcyYTJlZNgCBeACAQ&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=171&dest_type=country&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure"

## 4. Scrapin the top destinations ##

In [4]:
# Loading the Driver
driver = webdriver.Chrome("Driver/chromedriver.exe", options=options)
driver.get(url)
time.sleep(5) # These seconds  allow the page to fully load up.

# Get the Page Source to parse it with BS4
page_source = driver.page_source

# Closing the Driver
driver.quit()


C:\Users\tiago\AppData\Local\Temp\ipykernel_14944\1726225282.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("Driver/chromedriver.exe", options=options)


In [5]:
# Create a new BS4 object with the new page source
soup = BeautifulSoup(page_source, "lxml")

In [6]:
# Get the Top destinations divisions from the HTML as a list.
dest_div = soup.find("div", class_ ="ffa9856b86 c9c3c66a7d f7d4209a37").find("div", id = "filter_group_top_destinations_:R3cq:").find_all("div", class_ = "db29ecfbe2 c072c8cf10 c3f5dab487 a244b31530 a2d2507aff")

In [7]:
# Dicintionary to place all the top destinations and their specific code.
destinations = {}

# Looping all the top division to get their name and code
for dest in dest_div:
    destinations[dest.find("input").get("aria-label").split(":",1)[0]] = dest.find("input").get("name").split("=",1)[1]

## 5. Scraping the top 25 cities of the top destinations ##

With the code from the top destinations, we can loop the extended the url below with therespective code of the Top Destination. <br>
After it, we will scrape the cities and their respective code.

In [8]:
dest_url = "https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaLsBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKc7NCiBsACAdICJDYwM2Y1YTI5LTQ2ZjEtNDMzMy1hNWYzLTU0NTc1MDhlMmEzN9gCBeACAQ&aid=304142&ss=Portugal&ssne=Portugal&ssne_untouched=Portugal&lang=en-us&src=searchresults&dest_id=171&dest_type=country&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=top_destinations%3D"

In [9]:
cities = {} # City dict to put the top region and the cities code 
cities_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options) # Loading the Driver
for top in destinations:
    cities_driver.get(dest_url + str(destinations[top]))
    cities[top] = [] # Empty list to put the city and code dict
    time.sleep(10) # allow the page to fully load up.
    cities_page_source = cities_driver.page_source # Get the Page Source to parse it with BS4
    soup_cities = BeautifulSoup(cities_page_source, "lxml") # Create a new BS4 object with the new page source
    cities_div = soup_cities.find("div", class_ ="ffa9856b86 c9c3c66a7d f7d4209a37").find("div", id = "filter_group_uf_:R34q:").find_all("div", class_ = "db29ecfbe2 c072c8cf10 c3f5dab487 a244b31530 a2d2507aff") # Get the Cities divisions from the HTML as a list.
    for cit in cities_div:
        city = {}
        city [cit.find("input").get("aria-label").split(":",1)[0]] = cit.find("input").get("name").split("=",1)[1] # Get the city's code
        cities[top].append(city)
cities_driver.quit() # Closing the Driver

C:\Users\tiago\AppData\Local\Temp\ipykernel_14944\2113657281.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  cities_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options) # Loading the Driver


## 6. Creating a Dataframe with all the Top destinations and cities Combinations ##

Based on the top destinations and city code, we will create a dataframe with that information and the respective URL. <br>
We will then save that information to an XLSX file, so we don't have to scrape the infromation again, we can simply read a file with the links.

In [26]:
# Url to be formated below
# "https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaLsBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKc7NCiBsACAdICJDYwM2Y1YTI5LTQ2ZjEtNDMzMy1hNWYzLTU0NTc1MDhlMmEzN9gCBeACAQ&aid=304142&ss=Portugal&ssne=Portugal&ssne_untouched=Portugal&lang=en-us&src=searchresults&dest_id=171&dest_type=country&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=top_destinations%3D%3Buf%3D"

In [10]:
urls = [] #Empty list to append all dict. This will later be coverted to a dataframe

for top, city_list in cities.items(): # Looping all the top destinations
    for cidade in city_list:    # Looping all the cities in the top destinations
        url_dict = {}   # Dict will have the top destination, city and repective url. The URL below is a formatted string with the repective top destination and city code.
        url_dict["Top Destination"] = top # Top destiantion
        url_dict["City"] = list(cidade.keys())[0] # City
        # Add the respective URL city
        url_dict["Url"] = f"https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaLsBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKc7NCiBsACAdICJDYwM2Y1YTI5LTQ2ZjEtNDMzMy1hNWYzLTU0NTc1MDhlMmEzN9gCBeACAQ&aid=304142&ss=Portugal&ssne=Portugal&ssne_untouched=Portugal&lang=en-us&src=searchresults&dest_id=171&dest_type=country&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=top_destinations%3D{destinations[top]}%3Buf%3D{cidade[list(cidade.keys())[0]]}"
        urls.append(url_dict) # Append the cities URL to the list

In [11]:
df_cities = DataFrame(urls) # DF with the information Above
df_cities.to_excel("Urls/Destination and City URLs.xlsx", index=False) # Saving the DF to a Xlsx file.

## 7. Scraping the information of each Top Destination/City URL ##

### Important Note ###

Steps 3 to 6 can be skipped, and instead we can read the following XLSX file "Scraped Data/Destination and City URLs.xlsx". <br>
This file contains the URLs of all the pages to be scraped

In [29]:
df_read = read_excel("Urls/Destination and City URLs.xlsx") # Readt th DF
dic_cities = df_read.to_dict('records') # Change the DF to a list of Dictionaries

To the URL Below is the base URL for a City.<br>
Booking Website has a max of 40 pages displayed at one time. Each with 25 accomodations. <br>
Each page has a code ate the URL "offset=". <br>
We will loop through all fo the pages by changing the URL and get the summary information of the accomodations

In [80]:
url_base = "https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaLsBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKc7NCiBsACAdICJDYwM2Y1YTI5LTQ2ZjEtNDMzMy1hNWYzLTU0NTc1MDhlMmEzN9gCBeACAQ&aid=304142&ss=Portugal&ssne=Portugal&ssne_untouched=Portugal&lang=en-us&src=searchresults&dest_id=171&dest_type=country&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=top_destinations%3D%3Buf%3D-&offset="

### **Function:** get_summary_info(soup_object) ###

The function below will retrives the Name, the URL the Rating, the Revew Score, the number of reviews and the calssification fo the accomodation

In [82]:
def get_summary_info(soup_object):
    summary = {}
    try:
        summary["Name"] = soup_object.find("div", class_ = "fcab3ed991 a23c043802").text # Get the name of the accomodation
    except:
         summary["Name"] = ""
    try:
        summary["URL"] = soup_object.find("div", class_ = "d20f4628d0").find("div", class_ = "f9d4f2568d").find("div", class_ = "c90a25d457").find("a").get("href") # Get the URL of the accomodation
    except:
        summary["URL"] = ""
    try:
        summary["Rating"] = soup_object.find("div", class_ = "f919b8b3d5").find("div", class_ = "e4755bbd60").get("aria-label") # Get the rating the accomodation
    except:
        summary["Rating"] = ""
    try:
        summary["Review_Score"] = soup_object.find_all("div", class_ = "a1b3f50dcd ef8295f3e6 f7c6687c3d")[1].find("span").find("div", class_ = "b5cd09854e d10a6220b4").text # Get the review score of the accomodation
    except:
        summary["Review_Score"] = ""
    try:
        summary["Number_Reviews"] =  soup_object.find_all("div", class_ = "a1b3f50dcd ef8295f3e6 f7c6687c3d")[1].find("span").find("div", class_ = "d8eab2cf7f c90c0a70d3 db63693c62").text # Get the review score of the accomodation
    except:
        summary["Number_Reviews"] = ""
    try:
        summary["Classification"] =  soup_object.find_all("div", class_ = "a1b3f50dcd ef8295f3e6 f7c6687c3d")[1].find("span").find("div", class_ = "b5cd09854e f0d4d6a2f5 e46e88563a").get("aria-label").strip() # Get the classification score of the accomodation
    except:
        summary["Classification"] = ""
    return summary

In [25]:
# First Part we will get the number of pages per city we have available
city_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options) # Loading the Driver
for city in dic_cities:
    city_driver.get(city["Url"])
    time.sleep(5) # allow the page to fully load up.
    city_page_source = city_driver.page_source # Get the Page Source to parse it with BS4
    soup_city = BeautifulSoup(city_page_source, "lxml") # Create a new BS4 object with the new page source
    # This try and except, is only in case a city has less than 25 accomodations and only has one page.
    try:
        last_page = soup_city.find("div", class_="d7a0553560").find("ol", class_ = "a8b500abde").find_all("li", class_="f32a99c8d1")[-1].find("button").get("aria-label").strip() # This gets the number of the last page.
        city_places = [] # Dict to append the accomodations of the city
        for page in list(range(0,int(last_page)*25,25)): # THis will loop all the pages of the city
            city_driver.get(city["Url"]+"&offset="+str(page)) # Go to the page
            time.sleep(5) # load pgae
            city_page_source = city_driver.page_source # get HTML
            soup_city = BeautifulSoup(city_page_source, "lxml") #Parse HTML
            houses = [] # List to add the accomodations
            for house in soup_city.find_all("div", class_ = "a826ba81c4 fe821aea6c fa2f36ad22 afd256fc79 d08f526e0d ed11e24d01 ef9845d4b3 da89aeb942"): # loop all the accomodations in a page
                place = {} #dict to put the info
                place["Top_destination"] = city["Top Destination"] # Top dest
                place["City"] = city["City"] # City
                place_2 = get_summary_info(house) # Apply function described above
                final_place = place | place_2 # merge the 2 dict
                houses.append(final_place) # append final dic to the list
            city_places.extend(houses)
        df_cityplaces = DataFrame(city_places) #convert list of dict to DF
        df_cityplaces.to_excel(f"Scraped Data/Top- {city['Top Destination']} City- {city['City']}.xlsx", index=False) # Save to excel file
    # The except below works the exact way as the try above but only for the first page. This conditionwill only be ran if the city has less than 1 page
    except:
        city_places = []
        city_driver.get(city["Url"]+"&offset="+str(0))
        time.sleep(5)
        soup_city = BeautifulSoup(city_page_source, "lxml")
        houses = []
        for house in soup_city.find_all("div", class_ = "a826ba81c4 fe821aea6c fa2f36ad22 afd256fc79 d08f526e0d ed11e24d01 ef9845d4b3 da89aeb942"):
            place = {}
            place["Top_destination"] = city["Top Destination"]
            place["City"] = city["City"]
            place_2 = get_summary_info(house)
            final_place = place | place_2
            houses.append(final_place)
            city_places.extend(houses)
        df_cityplaces = DataFrame(city_places)
        df_cityplaces.to_excel(f"Scraped Data/Top- {city['Top Destination']} City- {city['City']}.xlsx", index=False)
city_driver.quit()

C:\Users\tiago\AppData\Local\Temp\ipykernel_14944\1483894870.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  city_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options) # Loading the Driver


## 8. Getting other top 25 cities on booking not in the top destinations ##

Some cities, like Lisbon, are not included in any of the top destinations.
We will get the total top 25 citties and then see what cities are not in any top destination. <br>
After it we will scrape the info of those cities using the code written in the step above.

In [6]:
# Loading the Driver
driver = webdriver.Chrome("Driver/chromedriver.exe", options=options)
driver.get(url)
time.sleep(10) # allow the page to fully load up.

# Get the Page Source to parse it with BS4
page_source = driver.page_source

# Closing the Driver
driver.quit()

C:\Users\tiago\AppData\Local\Temp\ipykernel_17112\3917965545.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("Driver/chromedriver.exe", options=options)


In [7]:
# Create a new BS4 object with the new page source
others_soup = BeautifulSoup(page_source, "lxml")

In [9]:
# Get the Top destinations divisions from the HTML as a list.
other_cities = others_soup.find("div", class_ ="ffa9856b86 c9c3c66a7d f7d4209a37").find("div", id = "filter_group_uf_:R34q:").find_all("div", class_ = "db29ecfbe2 c072c8cf10 c3f5dab487 a244b31530 a2d2507aff")

In [20]:
# Dicintionary to place all the top destinations and their specific code.
oth_city = {}

# Looping all the top division to get their name and code
for cit in other_cities:
        oth_city[cit.find("input").get("aria-label").split(":",1)[0]] = cit.find("input").get("name").split("=",1)[1]

In [71]:
other_urls = []
for other, cyti in oth_city.items(): # Looping all the top destinations
    url_dict = {}   # Dict will have the top destination, city and repective url. The URL below is a formatted string with the repective top destination and city code.
    url_dict["Top Destination"] = "Other"
    url_dict["City"] = other
    url_dict["Url"] = f"https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaLsBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKc7NCiBsACAdICJDYwM2Y1YTI5LTQ2ZjEtNDMzMy1hNWYzLTU0NTc1MDhlMmEzN9gCBeACAQ&aid=304142&ss=Portugal&ssne=Portugal&ssne_untouched=Portugal&lang=en-us&src=searchresults&dest_id=171&dest_type=country&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=uf%3D{str(cyti)}"
    other_urls.append(url_dict)

Filtering the other_urls to remove the cities that are in any of the top destinations

In [72]:
df_read = read_excel("Urls/Destination and City URLs.xlsx")
dic_cities = df_read.to_dict('records')

In [73]:
indexes = []
for index, dic in enumerate(other_urls):
    for city in dic_cities:
        if dic["City"] == city["City"]:
            try:
                indexes.append(index)
            except:
                None
                

In [74]:
# Removing the Duplicate Cities
indexes = list(set(indexes))

In [76]:
# Adding the cities in the top 25 cities that are not in any of  the top destinations
filtered_urls = []
for i, url in enumerate(other_urls):
    if i not in indexes:
        filtered_urls.append(url)

In [78]:
df_cities = DataFrame(filtered_urls) # DF with the information Above
df_cities.to_excel("Urls/Other Cities URLs.xlsx", index=False) # Saving the DF to a Xlsx file.

## 9. Scrapping the information of the Other Cities not in the Tope Destinations ##

In [87]:
df_read_2 = read_excel("Urls/Other Cities URLs.xlsx")
dic_cities_2 = df_read_2.to_dict('records')

In [81]:
url_base = "https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaLsBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKc7NCiBsACAdICJDYwM2Y1YTI5LTQ2ZjEtNDMzMy1hNWYzLTU0NTc1MDhlMmEzN9gCBeACAQ&aid=304142&ss=Portugal&ssne=Portugal&ssne_untouched=Portugal&lang=en-us&src=searchresults&dest_id=171&dest_type=country&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=top_destinations%3D%3Buf%3D-&offset="

The Process below is exactly the same as the one described above.

In [89]:
# First Part we will get the number of pages per city we have available
city_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options) # Loading the Driver
for city in dic_cities_2:
    city_driver.get(city["Url"])
    time.sleep(5) # allow the page to fully load up.
    city_page_source = city_driver.page_source # Get the Page Source to parse it with BS4
    soup_city = BeautifulSoup(city_page_source, "lxml") # Create a new BS4 object with the new page source
    try:
        last_page = soup_city.find("div", class_="d7a0553560").find("ol", class_ = "a8b500abde").find_all("li", class_="f32a99c8d1")[-1].find("button").get("aria-label").strip() # This gets the number of the last page.
        city_places = []
        for page in list(range(0,int(last_page)*25,25)):
            city_driver.get(city["Url"]+"&offset="+str(page))
            time.sleep(5)
            city_page_source = city_driver.page_source
            soup_city = BeautifulSoup(city_page_source, "lxml")
            houses = []
            for house in soup_city.find_all("div", class_ = "a826ba81c4 fe821aea6c fa2f36ad22 afd256fc79 d08f526e0d ed11e24d01 ef9845d4b3 da89aeb942"):
                place = {}
                place["Top_destination"] = city["Top Destination"]
                place["City"] = city["City"]
                place_2 = get_summary_info(house)
                final_place = place | place_2
                houses.append(final_place)
            city_places.extend(houses)
        df_cityplaces = DataFrame(city_places)
        df_cityplaces.to_excel(f"Scraped Data/Top- {city['Top Destination']} City- {city['City']}.xlsx", index=False)
    except:
        city_places = []
        city_driver.get(city["Url"]+"&offset="+str(0))
        time.sleep(5)
        soup_city = BeautifulSoup(city_page_source, "lxml")
        houses = []
        for house in soup_city.find_all("div", class_ = "a826ba81c4 fe821aea6c fa2f36ad22 afd256fc79 d08f526e0d ed11e24d01 ef9845d4b3 da89aeb942"):
            place = {}
            place["Top_destination"] = city["Top Destination"]
            place["City"] = city["City"]
            place_2 = get_summary_info(house)
            final_place = place | place_2
            houses.append(final_place)
            city_places.extend(houses)
        df_cityplaces = DataFrame(city_places)
        df_cityplaces.to_excel(f"Scraped Data/Top- {city['Top Destination']} City- {city['City']}.xlsx", index=False)
city_driver.quit()

C:\Users\tiago\AppData\Local\Temp\ipykernel_17112\1467691309.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  city_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options) # Loading the Driver


## 10. Merging all the saved XLSX into one Data Frame ##

### Important Note ###

Steps 3 to 9 can be skipped

Given that we have all the summary data, we can noe go to each of the urls and get a more detailed data about the property.<br>
First we will read all the XLSX file into a single python Dataframe

In [3]:
path = "Scraped Data"
files = glob.glob(path + "/*.xlsx") 

df_list = (read_excel(file) for file in files) # Read all the xlsx file the the folder
big_df = concat(df_list, ignore_index=True)

In [4]:
big_df.shape

(29174, 8)

In [5]:
big_df.head(2)

,Top_destination,City,Name,URL,Rating,Review_Score,Number_Reviews,Classification
0,Alentejo,Alcácer do Sal,Hotel da Barrosinha,https://www.booking.com/hotel/pt/albergaria-da...,4 out of 5,8.7,"1,385 reviews",Excellent
1,Alentejo,Alcácer do Sal,Alcácer 4Ever Palace,https://www.booking.com/hotel/pt/alcacer-4-eve...,NaN,8.0,304 reviews,Very Good


Since some properties can appear on multiple cities, we will remove the duplicates based on their URL

In [6]:
big_df = big_df[big_df['URL'].duplicated() == False]

In [7]:
final_dics = big_df.to_dict("records")

In [8]:
big_df.shape

(24910, 8)

## 11. Scraping more detailed information using each accomodation URL ##

Now that we have the URL of almost 25K accommodations, we will go to that URL and get more detailed information.

### **Function:** get_hotel_info(soup) ###

The function below will retrives the Name all the features listed below off of the accomodations Booking Webpage.

In [ ]:
def get_hotel_info(soup):
    hotel ={}
    try:
        hotel["Accommodation Type"] = soup.find("span", attrs={"data-testid": "property-type-badge"}).text # Get the Type of accomodation
    except:
        hotel["Accommodation Type"] = ""
    try:
        hotel["Rating Stars"] = len(soup.find("span", attrs={"data-testid": "rating-stars"}).find_all("span")) # Get the number of stars. only for hotels
    except:
        hotel["Rating Stars"] = ""
    try:
        hotel["Rating Squares"] = len(soup.find("span", attrs={"data-testid": "rating-squares"}).find_all("span")) # gets the number of squares, only for non hotels
    except:
        hotel["Rating Squares"] = ""
    try:
        hotel["Sustainable Level"] = soup.find("span", attrs={"class": "d8eab2cf7f cf9ebde7b2 be09c104ad"}).text # gets the sustainability level.
    except:
        hotel["Sustainable Level"] = ""
    try:
        hotel["Address"] = soup.find("span", attrs={"class": "hp_address_subtitle js-hp_address_subtitle jq_tooltip"}).text # Gets the address
    except:
        hotel["Address"] = ""
    try:
        hotel["Coordinates"] = soup.find("a", attrs={"id": "hotel_header"}).get("data-atlas-latlng") # Gets the coordinates
    except:
        hotel["Coordinates"] = ""
    try:
        hotel["Property Highlights"] = soup.find("span", attrs={"class": "ph-item-copy-heart"}).text # Gets the property highlights
    except:
        hotel["Property Highlights"] = ""
    try:
        hotel["Description"] = soup.find("div", attrs={"id": "property_description_content"}).text #Gets the decription of the accomodation
    except:
        hotel["Description"] = ""
    try:
        features = soup.find("div", attrs={"data-testid": "facility-list-most-popular-facilities"}).find_all("div") # Gets the most popular features/facilities
        for feature in features:
            hotel[feature.find("span", attrs={"class": "db312485ba"}).text] = 1 # Creates a dict, that if the accomodation has an ammenitie, it creates a dict. Key "name of the amenitie" Value = 1
    except:
        None
    try:
        categories = soup.find("div", attrs={"class", "a1b3f50dcd bdf0df2d01 bfcff4d0ca b2fe1a41c3 c1b465858f d46673fe81"}).find_all("div", class_ = "ccff2b4c43 cfc0860887") # Gets the rating of the categories
        for category in categories:
            hotel[category.find("span", attrs={"class", "d6d4671780"}).text] =  category.find("div", attrs={"class": "ee746850b6 b8eef6afe1"}).text
    except:
        None
    return hotel

In [11]:
hotel_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options)
final_info = []
for index, dic in enumerate(final_dics):
    hotel_driver.get(dic["URL"])
    time.sleep(3) # Load Page
    hotel_source = hotel_driver.page_source # get page source
    hotel_soup = BeautifulSoup(hotel_source, "lxml") #parse page source
    dic1 = {"Name": dic["Name"], "Top Destination": dic["Top_destination"], "City": dic["City"], "URL": dic["URL"], "Review_Score": dic["Review_Score"], "Number_Reviews": dic["Number_Reviews"], "Classification": dic["Classification"]} # Create Dict with already extracted and saved data
    dic2 = get_hotel_info(hotel_soup) # apply function described above
    comb_dic = dic1 | dic2 # Merge the 2 Dict
    final_info.append(comb_dic)
    if index % 500 == 0 or index == len(final_dics) - 1: # This will save the scraped information every 500 accomodations or until it reached the last one. In case IP gets block or some error occurs.
        DataFrame(final_info).to_excel(f"Final Information/Final Data Index - {index}.xlsx", index=False)
hotel_driver.quit()

C:\Users\tiago\AppData\Local\Temp\ipykernel_11916\1345040833.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  hotel_driver = webdriver.Chrome("Driver/chromedriver.exe", options=options)
